# Big Data Image Analysis

## zarrpaint
### Annotating directly into zarr arrays

TODO

zarrpaint - https://github.com/jni/zarpaint

Abigail McGovern & Juan Nunez-Iglesias

## dask-image segmentation pipeline

TODO

segmentation pipeline example

### Custom functionality

#### Exercise: write your own custom function for Dask

how to write your own functions to run custom functions with dask

Dask best practices

link to docs in napari

link to docs in dask array